# Module 2 - Working with Offline Feature Store 
## Features Store Dataset Extraction
**How to use Amazon SageMaker Feature Store to retrieve and share machine learning (ML) features in order to build feature sets that can be used for training a ML model.**

**Note:** Please set kernel to `Python 3 (Data Science)` and select instance to `ml.m5.4xlarge`

---

## Contents

1. [Overview](#Overview)
1. [Data Query](#Data-Query)
    1. [Look in S3 console](#Look-in-S3-console)
    1. [Query in Athena console](#Query-in-Athena-console)
    1. [Query using SageMaker SDK](#Query-using-SageMaker-SDK)
1. [Extract a feature set](#Extract-a-feature-set)    
1. [Optional queries and validations](#Optional-queries-and-validations)
    1. [Browse the set of offline store files in the S3 console](#Browse-the-set-of-offline-store-files-in-the-S3-console.)
    1. [See the Glue tables that are used for Athena queries](#See-the-Glue-tables-that-are-used-for-Athena-queries.)
    1. [Examine contents of a sample offline store Parquet file](#Examine-contents-of-a-sample-offline-store-Parquet-file.)
    1. [Count the rows in an offline store](#Count-the-rows-in-an-offline-store.)
    1. [Get a random sample of offline store rows](#Get-a-random-sample-of-offline-store-rows.)

---

## Overview

In previous module (`Module-1 Introduction to SageMaker Feature Store`), we demonstrated how to create multiple features groups inside a Amazon SageMaker Feature Store and ingest data into it.

In this notebook, we will illustrate how to retrieve the ingested features from the multiple feature groups and combine them to build feature sets that can be used to train a ML model. We will cover the following aspects:

* Look at data via S3 console (Offline feature store)
* Athena query for dataset extraction - shown via Athena console
* Athena query for dataset extraction (programmatically using SageMaker SDK)
* Extract training dataset and persist to S3
* Some additional queries to visualize a Parquet file, count the rows and get a random sample of offline store

---


## Data Query

Before starting, in order to get into Feature Store in SageMaker Studio in the left side menu you need to select the SageMaker Components and registries, then select Feature Store from the list of components and click on the Feature store button.

![Feature Store](../images/FS_0.png "Feature Store")

First, let's start by looking at the feature store data. 

Once the features groups are created (as demonstrated in `Module-1`), we should be able to see the three features groups - `orders`, `products` and `customers` from the SageMaker Studio UI.

![Features Groups](../images/FS1.png "Features Groups")

Double click on one feature group to open a new correspondent tab in Studio. Inside one particular feature group, we can check the location of the offline data by looking for s3 on the bottom right like in the following image. 

![Orders FG](../images/FS2.png "Orders FG")

### Look in S3 console

In a different tab please open S3 console in your AWS account. The location of the offline feature store data is in S3 inside the default S3 bucket and prefix will respect the following format:

s3://DEFAULT_BUCKET/sagemaker-feature-store/ACCOUNT_ID/sagemaker/REGION/offline-store/

![S3 Location](../images/S3_1.png "S3 Location")

We notice that in S3 we have a timestamp suffix added for each feature group name and this corresponds to what is displayed in Athena console as tables.

You can get the S3 URL from your environment after launching the cell referenced by: [Browse the set of offline store files in the S3 console.](#Browse-the-set-of-offline-store-files-in-the-S3-console.)

Inside each feature group, we have a `data` subdirectory followed by directories partitioned by year/month/day/hour of Parquet files.

![S3 Files](../images/S3_2.png "S3 Files")

You will can get a look into a sample Parquet file after launching the cell referenced by: [Examine contents of a sample offline store Parquet file](#Examine-contents-of-a-sample-offline-store-Parquet-file.)

---
### Query in Athena console

If it is for the first time we are launching Athena in AWS console we need to click on `Get Started` button and then before we run the first query we need to set up a query results location in Amazon S3. For simplicity, we can choose the same default SageMaker bucket that is used by Feature Store.

![Athena results location](../images/AthenaSetupMessage.png "Athena results location")

After setting the query results location, on the left panel we need to select the `AwsDataCatalog` as Data source and the `sagemaker_featurestore` as Database.

We can run now run a query for the offline feature store data in Athena. To select the entries from the orders feature group we use the following SQL query. You will need to replace the orders table name with the corresponded value from your environment.

```sql
select * from "<orders-feature-group-table-name>"
limit 20
```

![Athena Orders](../images/AthenaOrders.png "Athena Orders")

You can notice that Feature Store adds the `write_time`, `api_invocation_time` and `is_deleted` fields to the offline store.

Similarly, we can query the products and the customers feature group data by replacing the table name with the correspondent value. 

```sql
select * from "<products-feature-group-table-name>"
limit 20
select * from "<customers-feature-group-table-name>"
limit 20
```

Since our three features groups are related, we can do a join query by grouping all the information by product ID and by customer ID like here below. Please make sure you are replacing the table names with the corresponding values from your environment.

```sql
select *
FROM
  "<orders-feature-group-table-name>"
, "<products-feature-group-table-name>"  
, "<customers-feature-group-table-name>"
WHERE ("<orders-feature-group-table-name>"."customer_id" = "<customers-feature-group-table-name>"."customer_id")
AND ("<orders-feature-group-table-name>"."product_id" = "<products-feature-group-table-name>"."product_id")
limit 20
```

![Athena Join](../images/AthenaJoin.png "Athena Join")

---

### Query using SageMaker SDK

Extract the data from Feature Store using Athena query using SageMaker SDK

In [ ]:
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker import get_execution_role
import sagemaker
import logging
import boto3
import pandas as pd
import time
import re
import os
import sys
sys.path.append('..')
from utilities import Utils

In [ ]:
logger = logging.getLogger('__name__')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

#### Initialize default parameters

In [ ]:
sagemaker_session = sagemaker.Session()
account_id = sagemaker_session.account_id()
role = sagemaker.get_execution_role()
region = sagemaker_session.boto_region_name
default_bucket = sagemaker_session.default_bucket()
s3_client = boto3.client('s3', region_name=region)
query_results= 'sagemaker-featurestore-workshop'
prefix = 'sagemaker-feature-store'

#### Initialize boto3 runtime

In [ ]:
boto_session = boto3.Session(region_name=region)
sagemaker_client = boto_session.client(service_name='sagemaker', region_name=region)
featurestore_runtime = boto_session.client(service_name='sagemaker-featurestore-runtime', region_name=region)

feature_store_session = sagemaker.Session(boto_session=boto_session, 
                                          sagemaker_client=sagemaker_client, 
                                          sagemaker_featurestore_runtime_client=featurestore_runtime)

#### Retreive the orders, products and customers feature group names

In [ ]:
# Retreive FG names
%store -r customers_feature_group_name
%store -r products_feature_group_name
%store -r orders_feature_group_name
# Retreive FG row count
%store -r customers_count
%store -r products_count
%store -r orders_count

customers_fg = FeatureGroup(name=customers_feature_group_name, sagemaker_session=feature_store_session)  
products_fg = FeatureGroup(name=products_feature_group_name, sagemaker_session=feature_store_session)
orders_fg = FeatureGroup(name=orders_feature_group_name, sagemaker_session=feature_store_session)

#### Build Athena join query to combine the 3 features groups - `customers`, `products` & `orders`

In [ ]:
customers_query = customers_fg.athena_query()
customers_table = customers_query.table_name

products_query = products_fg.athena_query()
products_table = products_query.table_name

orders_query = orders_fg.athena_query()
orders_table = orders_query.table_name

In [ ]:
query_string = f'SELECT * FROM "{customers_table}", "{products_table}", "{orders_table}" ' \
               f'WHERE ("{orders_table}"."customer_id" = "{customers_table}"."customer_id") ' \
               f'AND ("{orders_table}"."product_id" = "{products_table}"."product_id")'
query_string

In [ ]:
output_location = f's3://{default_bucket}/{query_results}/query_results/'
print(f'Athena query output location: \n{output_location}')

#### Check to see if data is available in offline store.

In [ ]:
# Before extracting the data we need to check if the feature store was populated
offline_store_contents = None
while offline_store_contents is None:    
    customers_total_record_count = Utils.get_historical_record_count(customers_feature_group_name)
    products_total_record_count = Utils.get_historical_record_count(products_feature_group_name)
    orders_total_record_count = Utils.get_historical_record_count(orders_feature_group_name)
    if customers_total_record_count >= customers_count and \
        products_total_record_count >= products_count and \
        orders_total_record_count >= orders_count:
        logger.info('[Features are available in Offline Store!]')
        offline_store_contents = orders_total_record_count
    else:
        logger.info('[Waiting for data in Offline Store ...]')
        time.sleep(60)

#### Run Athena query and load the output as a Pandas dataframe.

In [ ]:
orders_query.run(query_string=query_string, output_location=output_location)
orders_query.wait()
joined_df = orders_query.as_dataframe()
joined_df.head()

In [ ]:
joined_df.columns

## Extract a feature set

**Note:** This extracted feature set will be used for model training in `Module-3`

##### Drop columns which are not needed for training

In [ ]:
model_df = joined_df.drop(['order_id', 
                           'customer_id', 
                           'product_id', 
                           'event_time', 
                           'write_time', 
                           'api_invocation_time', 
                           'is_deleted', 
                           'product_id.1', 
                           'event_time.1', 
                           'write_time.1', 
                           'api_invocation_time.1', 
                           'is_deleted.1', 
                           'customer_id.1', 
                           'purchase_amount',
                           'event_time.2', 
                           'n_days_since_last_purchase',
                           'write_time.2', 
                           'api_invocation_time.2', 
                           'is_deleted.2'], axis=1)

In [ ]:
model_df.head(5)

#### Write transformed features to local `data` directory

In [ ]:
model_df.to_csv('.././data/train/transformed.csv', index=False)

#### Copy file from local to S3

In [ ]:
boto3.Session().resource('s3').Bucket(default_bucket).Object(os.path.join(query_results, 'transformed.csv')).upload_file('.././data/train/transformed.csv')

## Optional queries and validations

### Browse the set of offline store files in the S3 console.

In [ ]:
customers_s3_console_url = Utils.get_offline_store_url(customers_feature_group_name)
products_s3_console_url = Utils.get_offline_store_url(products_feature_group_name)
orders_s3_console_url = Utils.get_offline_store_url(orders_feature_group_name)

logger.info('Review customers offline store partitioned data files here: '+customers_s3_console_url)
logger.info('\nReview products offline store partitioned data files here: '+products_s3_console_url)
logger.info('\nReview orders offline store partitioned data files here: '+orders_s3_console_url)

### See the Glue tables that are used for Athena queries.

In [ ]:
customers_glue_console_url = Utils.get_glue_table_url(customers_feature_group_name)
products_glue_console_url = Utils.get_glue_table_url(products_feature_group_name)
orders_glue_console_url = Utils.get_glue_table_url(orders_feature_group_name)

logger.info('To see the customers Glue table that was created for you, go here: '+customers_glue_console_url)
logger.info('\nTo see the products Glue table that was created for you, go here: '+products_glue_console_url)
logger.info('\nTo see the orders Glue table that was created for you, go here: '+orders_glue_console_url)

### Examine contents of a sample offline store Parquet file.

In [ ]:
# Let's have a look to the customers FG
customers_sample_filename = Utils.download_sample_offline_file(customers_feature_group_name)
logger.info('Downloaded sample Parquet file from offline store: '+customers_sample_filename+'\n')

customers_sample_df = pd.read_parquet(customers_sample_filename)
customers_sample_df.head()

### Count the rows in an offline store.

In [ ]:
# Get the number of rows from customers FG
customers_total_record_count = Utils.get_historical_record_count(customers_feature_group_name)
logger.info(f'Found {customers_total_record_count:,d} records in "{customers_feature_group_name}" feature group.')

### Get a random sample of offline store rows.

In [ ]:
# Get random rows from customers FG
customers_sample_df = Utils.sample(customers_feature_group_name, n=5)
customers_sample_df.head()